In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import MarianMTModel, MarianTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
from sacrebleu import corpus_bleu, corpus_chrf
import re

In [2]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"PyTorch version: {torch.__version__}")

CUDA available: True
GPU: NVIDIA GeForce GTX 1650
PyTorch version: 2.7.1+cu118


In [6]:
# ============================================
# STEP 1: Load Data (SHUFFLE!)
# ============================================
df = pd.read_csv("E:\\English-to-Hindi-Translation-Software\\data\\eng_hind.csv").dropna()
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(0.8 * len(df))
train_df = df[:train_size]
val_df = df[train_size:]

train_df.to_csv("E:\\English-to-Hindi-Translation-Software\\data\\train.csv", index=False)
val_df.to_csv("E:\\English-to-Hindi-Translation-Software\\data\\val.csv", index=False)
print(f"✅ Split: {len(train_df)} train, {len(val_df)} val")

✅ Split: 3653 train, 914 val


In [7]:
# ============================================
# STEP 2: Simple Dataset (NO FANCY STUFF)
# ============================================
class TranslationDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_length=96):  # ← 96 is sweet spot
        df = pd.read_csv(csv_path).dropna()
        self.eng = df["English"].tolist()
        self.hin = df["Hindi"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.eng)

    def __getitem__(self, index):
        src = self.eng[index]
        tgt = self.hin[index]

        # Standard encoding
        src_enc = self.tokenizer(
            src,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # CRITICAL: Use text_target for MarianMT!
        tgt_enc = self.tokenizer(
            text_target=tgt,  # ← This is KEY for Hindi!
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": src_enc["input_ids"].squeeze(0),
            "attention_mask": src_enc["attention_mask"].squeeze(0),
            "labels": tgt_enc["input_ids"].squeeze(0)
        }



In [8]:
# ============================================
# STEP 3: Setup (EXACT SETTINGS)
# ============================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# Datasets with max_length=96 (not 64, not 128)
train_dataset = TranslationDataset("E:\\English-to-Hindi-Translation-Software\\data\\train.csv", tokenizer, max_length=96)
val_dataset = TranslationDataset("E:\\English-to-Hindi-Translation-Software\\data\\val.csv", tokenizer, max_length=96)

# DataLoaders (smaller batch = better quality)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # ← 4, not 8!
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

print(f"✅ Ready: {len(train_loader)} train batches")

Device: cuda


e:\English-to-Hindi-Translation-Software\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Ready: 914 train batches


In [10]:
# ============================================
# STEP 4: OPTIMAL Training Settings
# ============================================
# CRITICAL: Lower learning rate for fine-tuning
optimizer = AdamW(
    model.parameters(),
    lr=3e-5,  # ← Lower than before (was 5e-5)
    weight_decay=0.01
)

# Add warmup + linear decay (prevents catastrophic forgetting)
num_epochs = 12  # ← Sweet spot for 4.5k dataset
num_training_steps = len(train_loader) * num_epochs
num_warmup_steps = len(train_loader) * 2  # ← 2 epochs warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

best_val_loss = float('inf')
patience = 3  # Early stopping
patience_counter = 0

print(f"Training for {num_epochs} epochs.")

Training for 12 epochs.


In [11]:
# ============================================
# STEP 5: Training Loop with Early Stopping
# ============================================
for epoch in range(num_epochs):
    # === TRAINING ===
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        # Gradient clipping (prevents explosion)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()  # ← Update learning rate

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # === VALIDATION ===
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    current_lr = scheduler.get_last_lr()[0]

    print(f"Epoch {epoch+1} | Train: {avg_train_loss:.4f} | Val: {avg_val_loss:.4f} | LR: {current_lr:.2e}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0

        model.save_pretrained("./models/best_helsinki_model")
        tokenizer.save_pretrained("./models/best_helsinki_model")
        print(f"Best model saved! (Val Loss: {avg_val_loss:.4f})")
    else:
        patience_counter += 1

    # Early stopping
    if patience_counter >= patience:
        print(f"\nEarly stopping at epoch {epoch+1} (no improvement for {patience} epochs)")
        break

print("\nTraining complete!")


Epoch 1/12: 100%|██████████| 914/914 [03:59<00:00,  3.81it/s] 


Epoch 1 | Train: 1.5873 | Val: 0.4532 | LR: 1.50e-05


e:\English-to-Hindi-Translation-Software\.venv\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Best model saved! (Val Loss: 0.4532)


Epoch 2/12: 100%|██████████| 914/914 [02:43<00:00,  5.57it/s]


...

Epoch 8 | Train: 0.0705 | Val: 0.3067 | LR: 1.20e-05


Epoch 9/12: 100%|██████████| 914/914 [02:42<00:00,  5.61it/s]


Epoch 9 | Train: 0.0579 | Val: 0.3090 | LR: 9.00e-06


Epoch 10/12: 100%|██████████| 914/914 [02:42<00:00,  5.61it/s]


Epoch 10 | Train: 0.0485 | Val: 0.3093 | LR: 6.00e-06

Early stopping at epoch 10 (no improvement for 3 epochs)

Training complete!


In [12]:
# ============================================
# STEP 6: Inference with Proper Settings
# ============================================
def translate(text):
    """Fixed inference matching training settings"""
    model.eval()
    with torch.no_grad():
        # Encode with same max_length as training
        inputs = tokenizer(
            text,
            return_tensors="pt",
            max_length=96,  # ← Match training!
            truncation=True,
            padding=True
        ).to(device)

        # Generate with proper settings
        outputs = model.generate(
            **inputs,
            max_length=96,      # ← Match training!
            num_beams=4,        # ← Lower beams = less repetition
            length_penalty=1.0,  # ← Balanced length
            early_stopping=True,
            no_repeat_ngram_size=2,  # ← Prevent loops
            repetition_penalty=1.2   # ← Avoid repetition
        )

        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # F3: Fix section numbers
        sections = re.findall(r'Section\s+(\d+)', text, re.IGNORECASE)
        if sections:
            translation = re.sub(r'धारा\s+\d+', f'धारा {sections[0]}', translation)

        return translation



In [14]:

# ============================================
# STEP 7: Test Translations
# ============================================
print("\n" + "="*60)
print("TESTING BEST MODEL")
print("="*60)

# Load best model
model = MarianMTModel.from_pretrained("./models/best_helsinki_model").to(device)
tokenizer = MarianTokenizer.from_pretrained("./models/best_helsinki_model")

test_cases = [
    "Parking is not allowed.",
    "Unauthorized entry is prohibited.",
    "This notice is issued under Section 144 of the Criminal Procedure Code",
    "The authorities announced a new curfew order.",
    "The policy will be implemented immediately.",
    "Smoking is strictly forbidden in public places."
]

for text in test_cases:
    result = translate(text)
    print(f"\nEN: {text}")
    print(f"HI: {result}")



TESTING BEST MODEL


e:\English-to-Hindi-Translation-Software\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



EN: Parking is not allowed.
HI: पार्किंग की अनुमति नहीं है।

EN: Unauthorized entry is prohibited.
HI: अनधिकृत प्रवेश प्रतिबंधित है।

EN: This notice is issued under Section 144 of the Criminal Procedure Code
HI: यह सूचना दंड प्रक्रिया संहिता की धारा 144 के अंतर्गत जारी की गई है

EN: The authorities announced a new curfew order.
HI: अधिकारियों ने नया कर्फ्यू आदेश घोषित किया।

EN: The policy will be implemented immediately.
HI: नीति तुरंत लागू की जाएगी।

EN: Smoking is strictly forbidden in public places.
HI: सार्वजनिक स्थानों पर धूम्रपान सख्ती से वर्जित है।


In [18]:
# Load validation data
val_data = pd.read_csv("E:\\English-to-Hindi-Translation-Software\\data\\val.csv").dropna()
sources = val_data['English'].tolist()
references = val_data['Hindi'].tolist()

print(f"\nEvaluating on {len(sources)} validation samples...")
print("This may take a few minutes...\n")

# Generate predictions for ALL validation data
predictions = []
for i, src in enumerate(tqdm(sources, desc="Generating translations")):
    pred = translate(src)
    predictions.append(pred)

    # Show progress every 100 samples
    if (i + 1) % 100 == 0:
        print(f"  Processed {i+1}/{len(sources)} samples")

# Calculate BLEU score
bleu = corpus_bleu(predictions, [references])
chrf = corpus_chrf(predictions, [references])

print("\n" + "="*60)
print("📊 FINAL EVALUATION RESULTS")
print("="*60)
print(f"✅ BLEU Score:  {bleu.score:.2f}")
print(f"✅ chrF Score:  {chrf.score:.2f}")
print(f"✅ Precision:   {bleu.precisions[0]:.2f}% (1-gram)")
print(f"✅ Precision:   {bleu.precisions[1]:.2f}% (2-gram)")
print(f"✅ Precision:   {bleu.precisions[2]:.2f}% (3-gram)")
print(f"✅ Precision:   {bleu.precisions[3]:.2f}% (4-gram)")
print("="*60)

# Additional analysis
print("\n📋 QUALITATIVE ANALYSIS (Sample Translations):\n")

# Show 10 random samples
import random
random.seed(42)
sample_indices = random.sample(range(len(sources)), min(10, len(sources)))

for idx in sample_indices:
    print(f"Example {idx+1}:")
    print(f"  Source:      {sources[idx]}")
    print(f"  Reference:   {references[idx]}")
    print(f"  Prediction:  {predictions[idx]}")
    print()

# F1: Formality Analysis
print("="*60)
print("📝 F1: FORMALITY ANALYSIS")
print("="*60)

formal_keywords = ['वर्जित', 'निषिद्ध', 'प्राधिकरण', 'सूचना', 'धारा', 'अधिसूचना', 'तत्काल']
formal_count = 0

for pred in predictions:
    if any(kw in pred for kw in formal_keywords):
        formal_count += 1

formality_pct = (formal_count / len(predictions)) * 100
print(f"✅ Formal style usage: {formality_pct:.1f}%")
print(f"   ({formal_count}/{len(predictions)} translations contain formal keywords)")

# F3: Number Preservation Analysis
print("\n" + "="*60)
print("🔢 F3: NUMBER PRESERVATION ANALYSIS")
print("="*60)

number_errors = 0
total_with_numbers = 0

for src, pred in zip(sources, predictions):
    src_numbers = set(re.findall(r'\b\d+\b', src))
    if src_numbers:
        total_with_numbers += 1
        pred_numbers = set(re.findall(r'\b\d+\b', pred))
        if src_numbers != pred_numbers:
            number_errors += 1

if total_with_numbers > 0:
    number_accuracy = ((total_with_numbers - number_errors) / total_with_numbers) * 100
    print(f"✅ Number preservation accuracy: {number_accuracy:.1f}%")
    print(f"   ({total_with_numbers - number_errors}/{total_with_numbers} correctly preserved)")
    print(f"   Errors: {number_errors}")
else:
    print("⚠️ No numbers found in validation set")

# Terminology Consistency Check
print("\n" + "="*60)
print("📖 F2: TERMINOLOGY CONSISTENCY")
print("="*60)

term_mapping = {
    'prohibited': ['वर्जित', 'निषिद्ध', 'प्रतिबंधित'],
    'notice': ['सूचना', 'नोटिस'],
    'section': ['धारा'],
    'authority': ['प्राधिकरण', 'अधिकारी'],
    'immediately': ['तुरंत', 'तत्काल']
}

consistency_count = 0
total_terms = 0

for src, pred in zip(sources, predictions):
    for eng_term, hindi_options in term_mapping.items():
        if eng_term in src.lower():
            total_terms += 1
            if any(hin in pred for hin in hindi_options):
                consistency_count += 1

if total_terms > 0:
    consistency_pct = (consistency_count / total_terms) * 100
    print(f"✅ Terminology consistency: {consistency_pct:.1f}%")
    print(f"   ({consistency_count}/{total_terms} terms correctly translated)")
else:
    print("⚠️ No key terms found in validation set")

# Save detailed results
print("\n" + "="*60)
print("💾 SAVING DETAILED RESULTS")
print("="*60)

results_df = pd.DataFrame({
    'English': sources,
    'Reference_Hindi': references,
    'Predicted_Hindi': predictions
})

results_df.to_csv("evaluation_results.csv", index=False)
print("✅ Saved detailed results to: evaluation_results.csv")

# Summary statistics
print("\n" + "="*60)
print("📈 SUMMARY STATISTICS")
print("="*60)
print(f"Total validation samples: {len(sources)}")
print(f"Average source length: {sum(len(s.split()) for s in sources)/len(sources):.1f} words")
print(f"Average reference length: {sum(len(r.split()) for r in references)/len(references):.1f} words")
print(f"Average prediction length: {sum(len(p.split()) for p in predictions)/len(predictions):.1f} words")

print("\n" + "="*60)
print("🎯 FINAL VERDICT")
print("="*60)

if bleu.score >= 35:
    print("🏆 EXCELLENT! Production-ready quality!")
elif bleu.score >= 25:
    print("✅ GOOD! Suitable for deployment with monitoring.")
elif bleu.score >= 15:
    print("⚠️ FAIR. Consider more training or data.")
else:
    print("❌ POOR. Needs significant improvement.")

print("\n" + "="*60)


Evaluating on 914 validation samples...
This may take a few minutes...



Generating translations:  11%|█         | 100/914 [00:35<04:58,  2.73it/s]

  Processed 100/914 samples


Generating translations:  22%|██▏       | 200/914 [01:08<04:07,  2.88it/s]

  Processed 200/914 samples


...

  Processed 800/914 samples


Generating translations:  99%|█████████▊| 901/914 [05:00<00:03,  3.50it/s]

  Processed 900/914 samples


Generating translations: 100%|██████████| 914/914 [05:04<00:00,  3.00it/s]



📊 FINAL EVALUATION RESULTS
✅ BLEU Score:  41.76
✅ chrF Score:  64.91
✅ Precision:   69.12% (1-gram)
✅ Precision:   49.18% (2-gram)
✅ Precision:   35.97% (3-gram)
✅ Precision:   27.16% (4-gram)

📋 QUALITATIVE ANALYSIS (Sample Translations):

Example 655:
  Source:      Diversion signs have been installed for commuters
  Reference:   यात्रियों के लिए डायवर्जन संकेत लगाए गए हैं
  Prediction:  यात्रियों के लिए रूपांतरण संकेत लगाए गए हैं

Example 115:
  Source:      Original documents must be produced for comparison when required by the court
  Reference:   मूल दस्तावेज न्यायालय द्वारा आवश्यक होने पर तुलना के लिए प्रस्तुत करने होंगे
  Prediction:  न्यायालय द्वारा आवश्यक होने पर मूल दस्तावेजों की तुलना की जानी चाहिए

Example 26:
  Source:      Jan Dhan account holders get free RuPay debit card and accident insurance. Zero balance accounts allowed.
  Reference:   जन धन खाताधारकों को निःशुल्क रुपे डेबिट कार्ड एवं दुर्घटना बीमा मिलेगा। शून्य बैलेंस खाते अनुमत हैं।
  Prediction:  जनन खाता धारको